In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
%matplotlib inline
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import pandas as pd
import random

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
#this next line is only needed in iPython notebooks
%matplotlib inline 
import math
import matplotlib.font_manager as fm
import matplotlib.ticker as mtick
font = fm.FontProperties(size = 12)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
def hide_spines(intx=False,inty=False):
    """Hides the top and rightmost axis spines from view for all active
    figures and their respective axes."""

    # Retrieve a list of all current figures.
    figures = [x for x in matplotlib._pylab_helpers.Gcf.get_all_fig_managers()]
    if (plt.gca().get_legend()):
        plt.setp(plt.gca().get_legend().get_texts(), fontproperties=font) 
    for figure in figures:
        # Get all Axis instances related to the figure.
        for ax in figure.canvas.figure.get_axes():
            # Disable spines.
            ax.spines['right'].set_color('none')
            ax.spines['top'].set_color('none')
            # Disable ticks.
            ax.xaxis.set_ticks_position('bottom')
            ax.yaxis.set_ticks_position('left')
           # ax.xaxis.set_major_formatter(mtick.FuncFormatter(lambda v,_: ("10$^{%d}$" % math.log(v,10)) ))
            for label in ax.get_xticklabels() :
                label.set_fontproperties(font)
            for label in ax.get_yticklabels() :
                label.set_fontproperties(font)
            #ax.set_xticklabels(ax.get_xticks(), fontproperties = font)
            ax.set_xlabel(ax.get_xlabel(), fontproperties = font)
            ax.set_ylabel(ax.get_ylabel(), fontproperties = font)
            ax.set_title(ax.get_title(), fontproperties = font)
            if (inty):
                ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('%d'))
            if (intx):
                ax.xaxis.set_major_formatter(mtick.FormatStrFormatter('%d'))
def show(nm,a=0,b=0,show=1):
    hide_spines(a,b)
    if (len(nm)>0):
        plt.savefig(nm,bbox_inches='tight');
    if show:
        plt.show()
    else:
        plt.close()

The equation for the damped pendulum is $$ \frac{d ^2\theta}{dt} + \frac{b}{m} \frac{d \theta}{dt} + \frac{g}{l} \sin \theta = 0.$$

Defining $y(t) = \theta'(t)$, we get the system
$$ y'(t) = -\frac{b}{m} y(t) - \frac{g}{l} \sin \theta,$$
$$ \theta'(t) = y(t).$$

In [ ]:
#define function for RHS
b = 0.1
l = 2.5
g = 9.81
m = 0.25

print(b/m,g/l)
RHS = lambda t,x: [-b/m * x[0] - g/l*math.sin(x[1]), x[0]]

In [ ]:
#y0=[-4.283866414839002,math.pi/1.05] stops at the top
solution = solve_ivp(fun=RHS, y0=[-4.283866414838997,math.pi/1.05], t_span=[0,25], max_step=0.001, method="RK45", first_step = 0.001)

In [ ]:
def get_sample(runs,points):
    y_vect = np.zeros(runs*points)
    x_mat = np.zeros((runs*points, 28))
    basis_dict = lambda pos, vel: np.array([pos, pos**2, pos**3, pos**4, pos**5, math.cos(pos), math.sin(pos), math.sin(pos)**2, math.cos(pos)**2,
                                            vel, vel**2, vel**3, vel**4, vel**5, math.cos(vel), math.sin(vel), math.sin(vel)**2, math.cos(vel)**2,
                                            pos*vel, (pos**2)*vel, pos*vel**2, (pos**2)*(vel**2), (pos**3)*(vel), (pos)*(vel**3), (pos**2)*(vel**3), (pos**3)*(vel**2),
                                            math.cos(pos*vel), math.sin(pos*vel)])
    count = 0
    
    for run in range(runs):
        init_vel = random.uniform(a=-10,b=10)
        init_pos = random.uniform(a=-np.pi, b=np.pi)
        solution = solve_ivp(fun=RHS, y0=[init_vel, init_pos], t_span=[0,25], max_step=0.01, method="RK45")
        steps = solution.t.size
        chosen_points = np.random.randint(low=1, high=steps-1,size=points)
        velocity = solution.y[0,:].copy()
        position = np.arctan2(np.sin(solution.y[1,:]),np.cos(solution.y[1,:]))
        for point in range(points):
            y_vect[count] = ((velocity[chosen_points[point]] - velocity[chosen_points[point]-1])/
                      (solution.t[chosen_points[point]] - solution.t[chosen_points[point]-1]))
            x_mat[count,:] = basis_dict(position[chosen_points[point]],
                                        velocity[chosen_points[point]])
            count += 1
    return y_vect, x_mat

In [ ]:
b = 0.1
l = 2.5
g = 9.81
m = 0.25
y,X = get_sample(100,10)

In [ ]:
#plt.scatter(y, -b/m*X[:,8]- g/l*X[:,5])
#plt.scatter(X[:,0],X[:,1],s=1)
plt.figure(figsize=(8,8))
plt.subplot(331)
plt.scatter(X[:,0], y, s=1, marker=".", c="k")
plt.ylabel("$\\ddot{\\theta}$")
plt.xlabel("$\\theta$")
plt.subplot(332)
plt.scatter(X[:,2], y, s=1, marker=".", c="k")
plt.xlabel("$\\theta^3$")
plt.subplot(333)
plt.scatter(X[:,6], y, s=1, marker=".", c="k")
plt.xlabel("$\\sin \\theta$")

#row 2
plt.subplot(334)
plt.scatter(X[:,9], y, s=1, marker=".", c="k")
plt.ylabel("$\\ddot{\\theta}$")
plt.xlabel("$\\dot\\theta$")
plt.subplot(335)
plt.scatter(X[:,10], y, s=1, marker=".", c="k")
plt.xlabel("$\\dot{\\theta}^2$")
plt.subplot(336)
plt.scatter(X[:,19], y, s=1, marker=".", c="k")
plt.xlabel("$\\theta^2\\dot{\\theta}$")
#row 3
plt.subplot(337)
plt.scatter(b/m*X[:,9]+g/l*X[:,6], y, s=1, marker=".", c="k")
plt.ylabel("$\\ddot{\\theta}$")
plt.xlabel("$\\frac{b}{m}\\dot\\theta + \\frac{g}{L}\\sin \\theta $")
plt.subplot(338)
plt.scatter(b/m*X[:,9]+g/l*X[:,0]-g/l/6*X[:,2], y, s=1, marker=".", c="k")
plt.xlabel("$\\frac{b}{m}\\dot\\theta+\\frac{g }{L}\\theta-\\frac{g }{6 L}\\theta^3$")
plt.subplot(339)
plt.scatter(b/m*X[:,9]+g/l*X[:,0]-g/l/6*X[:,2]+g*X[:,4]/120/l, y, s=1, marker=".", c="k")
plt.xlabel("$\\frac{b}{m}\\dot\\theta-\\frac{g \\theta}{L}+\\frac{g \\theta^3}{6 L}-\\frac{g \\theta^5}{120 L}$")
plt.tight_layout()
#plt.subplots_adjust(bottom=0.9,top=1) #, right=1.8, top=0.9)
show("pendulum_scatter.pdf")

In [ ]:
from sklearn.linear_model import LassoCV
reg = LassoCV(cv=5, random_state=0, normalize=True).fit(X, y)
reg.score(X, y) 
print("Sin coeff should be", -g/l, "it is",reg.coef_[6])
print("Velocity coeff should be", -b/m, "it is",reg.coef_[9])
print("Velocity coeff should be", 0, "it is",reg.coef_[19])
lasso = reg.coef_.copy()
print(lasso)

In [ ]:
from sklearn.linear_model import ElasticNetCV
reg = ElasticNetCV(cv=5, random_state=0, normalize=True,l1_ratio=0.9).fit(X, y)
reg.score(X, y) 
print("Sin coeff should be", -g/l, "it is",reg.coef_[6])
print("Velocity coeff should be", -b/m, "it is",reg.coef_[9])
el9 = reg.coef_.copy()
print(el9)

In [ ]:
from sklearn.linear_model import ElasticNetCV
reg = ElasticNetCV(cv=5, random_state=0, normalize=True,l1_ratio=0.5).fit(X, y)
reg.score(X, y) 
print("Sin coeff should be", -g/l, "it is",reg.coef_[6])
print("Velocity coeff should be", -b/m, "it is",reg.coef_[9])
el5 = reg.coef_.copy()
print(el5)

In [ ]:
from sklearn.linear_model import RidgeCV
reg = RidgeCV(cv=5, normalize=True).fit(X, y)
reg.score(X, y) 
print("Sin coeff should be", -g/l, "it is",reg.coef_[6])
print("Velocity coeff should be", -b/m, "it is",reg.coef_[9])
ridge = reg.coef_.copy()
print(ridge)

In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X, y)
reg.score(X, y) 
print("Sin coeff should be", -g/l, "it is",reg.coef_[6])
print("Velocity coeff should be", -b/m, "it is",reg.coef_[9])
ls = reg.coef_.copy()
print(ls)

In [ ]:
tval = 0*ls
tval[6] = -g/l
tval[9] = -b/m
my_xticks = np.array(np.arange(tval.size), dtype=str)
my_xticks[0] = "$\\theta$"
my_xticks[6] = "$\\sin\\theta$"
my_xticks[9] = "$\\dot\\theta$"
tmp = np.array(("$\\theta$", "$\\theta^2$", "$\\theta^3$", "$\\theta^4$", 
                "$\\theta^5$", "$\\cos\\theta$", "$\\sin\\theta$", 
                "$\\sin^2\\theta$", "$\\cos^2\\theta$","$\\dot\\theta$", 
                "$\\dot\\theta^2$", "$\\dot\\theta^3$", "$\\dot\\theta^4$", 
                "$\\dot\\theta^5$", "$\\cos\\dot\\theta$", "$\\sin\\dot\\theta$", 
                "$\\sin^2\\dot\\theta$", "$\\cos^2\\dot\\theta$",
                "$\\theta\\dot\\theta$", "$\\theta^2\\dot\\theta$", 
                "$\\theta\\dot\\theta^2$", "$\\theta^2 (\\dot\\theta)^2$", 
                "$(\\theta^3)*(\\dot\\theta)$", "$(\\theta)*(\\dot\\theta^3)$", 
                "$\\theta^2 \\dot\\theta^3$", "$(\\theta^3)*(\\dot\\theta^2)$",
                "$\\cos(\\theta*\\dot\\theta)$", "$\\sin(\\theta*\\dot\\theta)$"))
my_xticks[0:21]=tmp[0:21]
#print(my_xticks)
plt.figure(figsize=(3.2*3,3))
plt.plot(np.abs(tval[0:21]), 'o',label="true",fillstyle='none')
plt.plot(np.abs(lasso[0:21]), '^', label="lasso",fillstyle='none')
plt.plot(np.abs(el9[0:21]), '*', label="elnet $\\alpha=0.9$")
plt.plot(np.abs(el5[0:21]), 'p',label="elnet $\\alpha=0.5$",fillstyle='none')
plt.plot(np.abs(ridge[0:21]), '>', label="ridge",fillstyle='none')
plt.plot(np.abs(ls[0:21]), '.', label="least squares")
plt.xticks(np.arange(tval.size)[0:21],my_xticks[0:21],rotation=75);
plt.ylabel("|coefficient|")
plt.legend(loc="best")
show("pendulum_coeff.pdf")


plt.plot(np.abs(tval[0:21]), 'o',label="true",fillstyle='none')
plt.plot(np.abs(lasso[0:21]), '^', label="lasso")
plt.plot(np.abs(el9[0:21]), '*', label="elnet $\\alpha=0.9$")
plt.plot(np.abs(el5[0:21]), 'p',label="elnet $\\alpha=0.5$")
plt.plot(np.abs(ridge[0:21]), '>', label="ridge")
plt.semilogy(np.abs(ls[0:21]), '.', label="least squares")
plt.xticks(np.arange(tval.size)[0:21],my_xticks[0:21],rotation=75);
plt.ylabel("|coefficient|")
plt.ylim([1e-4,3])
plt.legend(loc=6)
show("pendulum_coeff_log.pdf")

In [ ]:
def get_sample_with_noise(runs,points,noise=0.001):
    y_vect = np.zeros(runs*points)
    x_mat = np.zeros((runs*points, 28))
    basis_dict = lambda pos, vel: np.array([pos, pos**2, pos**3, pos**4, pos**5, math.cos(pos), math.sin(pos), math.sin(pos)**2, math.cos(pos)**2,
                                            vel, vel**2, vel**3, vel**4, vel**5, math.cos(vel), math.sin(vel), math.sin(vel)**2, math.cos(vel)**2,
                                            pos*vel, (pos**2)*vel, pos*vel**2, (pos**2)*(vel**2), (pos**3)*(vel), (pos)*(vel**3), (pos**2)*(vel**3), (pos**3)*(vel**2),
                                            math.cos(pos*vel), math.sin(pos*vel)])
    count = 0
    for run in range(runs):
        init_vel = random.uniform(a=-10,b=10)
        init_pos = random.uniform(a=-np.pi, b=np.pi)
        solution = solve_ivp(fun=RHS, y0=[init_vel, init_pos], t_span=[0,25], max_step=0.01, method="RK45")
        steps = solution.t.size
        chosen_points = np.random.randint(low=1, high=steps-1,size=points)
        velocity = solution.y[0,:] +np.random.normal(loc=0, scale=noise, size=steps)
        position = np.arctan2(np.sin(solution.y[1,:]),np.cos(solution.y[1,:])) +np.random.normal(loc=0, scale=noise, size=steps)
        for point in range(points):
            y_vect[count] = ((velocity[chosen_points[point]] - velocity[chosen_points[point]-1])/
                      (solution.t[chosen_points[point]] - solution.t[chosen_points[point]-1]))
            x_mat[count,:] = basis_dict(position[chosen_points[point]],
                                        velocity[chosen_points[point]])
            count += 1
    return y_vect, x_mat

In [ ]:
y,X = get_sample_with_noise(100,10,0.01)

In [ ]:
#plt.scatter(y, -b/m*X[:,8]- g/l*X[:,5])
#plt.scatter(X[:,0],X[:,1],s=1)
plt.figure(figsize=(8,8))
plt.subplot(331)
plt.scatter(X[:,0], y, s=1, marker=".", c="k")
plt.ylabel("$\\ddot{\\theta}$")
plt.xlabel("$\\theta$")
plt.subplot(332)
plt.scatter(X[:,2], y, s=1, marker=".", c="k")
plt.xlabel("$\\theta^3$")
plt.subplot(333)
plt.scatter(X[:,6], y, s=1, marker=".", c="k")
plt.xlabel("$\\sin \\theta$")

#row 2
plt.subplot(334)
plt.scatter(X[:,9], y, s=1, marker=".", c="k")
plt.ylabel("$\\ddot{\\theta}$")
plt.xlabel("$\\dot\\theta$")
plt.subplot(335)
plt.scatter(X[:,10], y, s=1, marker=".", c="k")
plt.xlabel("$\\dot{\\theta}^2$")
plt.subplot(336)
plt.scatter(X[:,19], y, s=1, marker=".", c="k")
plt.xlabel("$\\theta^2\\dot{\\theta}$")
#row 3
plt.subplot(337)
plt.scatter(b/m*X[:,9]+g/l*X[:,6], y, s=1, marker=".", c="k")
plt.ylabel("$\\ddot{\\theta}$")
plt.xlabel("$\\frac{b}{m}\\dot\\theta + \\frac{g}{L}\\sin \\theta $")
plt.subplot(338)
plt.scatter(b/m*X[:,9]+g/l*X[:,0]-g/l/6*X[:,2], y, s=1, marker=".", c="k")
plt.xlabel("$\\frac{b}{m}\\dot\\theta+\\frac{g }{L}\\theta-\\frac{g }{6 L}\\theta^3$")
plt.subplot(339)
plt.scatter(b/m*X[:,9]+g/l*X[:,0]-g/l/6*X[:,2]+g*X[:,4]/120/l, y, s=1, marker=".", c="k")
plt.xlabel("$\\frac{b}{m}\\dot\\theta-\\frac{g \\theta}{L}+\\frac{g \\theta^3}{6 L}-\\frac{g \\theta^5}{120 L}$")
plt.tight_layout()
#plt.subplots_adjust(bottom=0.9,top=1) #, right=1.8, top=0.9)
show("pendulum_scatter_noise.pdf")

In [ ]:
from sklearn.linear_model import LassoCV
reg = LassoCV(cv=5, random_state=0, normalize=True).fit(X, y)
reg.score(X, y) 
print("Sin coeff should be", -g/l, "it is",reg.coef_[6])
print("Velocity coeff should be", -b/m, "it is",reg.coef_[9])
reg.coef_

In [ ]:
from sklearn.linear_model import ElasticNetCV
reg = ElasticNetCV(cv=5, random_state=0, normalize=True,l1_ratio=0.9).fit(X, y)
print(reg.score(X, y) )
print("Sin coeff should be", -g/l, "it is",reg.coef_[6])
print("Velocity coeff should be", -b/m, "it is",reg.coef_[9])
reg.coef_

In [ ]:
from sklearn.linear_model import ElasticNetCV
reg = ElasticNetCV(cv=5, random_state=0, normalize=True,l1_ratio=0.5).fit(X, y)
reg.score(X, y) 
print("Sin coeff should be", -g/l, "it is",reg.coef_[6])
print("Velocity coeff should be", -b/m, "it is",reg.coef_[9])
el5 = reg.coef_.copy()
print(el5)

In [ ]:
from sklearn.linear_model import RidgeCV
reg = RidgeCV(cv=5, normalize=True).fit(X, y)
reg.score(X, y) 
print("Sin coeff should be", -g/l, "it is",reg.coef_[6])
print("Velocity coeff should be", -b/m, "it is",reg.coef_[9])
ridge = reg.coef_.copy()
print(ridge)

In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X, y)
reg.score(X, y) 
print("Sin coeff should be", -g/l, "it is",reg.coef_[6])
print("Velocity coeff should be", -b/m, "it is",reg.coef_[9])
ls = reg.coef_.copy()
print(ls)

In [ ]:
tval = 0*ls
tval[6] = -g/l
tval[9] = -b/m
my_xticks = np.array(np.arange(tval.size), dtype=str)
my_xticks[0] = "$\\theta$"
my_xticks[6] = "$\\sin\\theta$"
my_xticks[9] = "$\\dot\\theta$"
tmp = np.array(("$\\theta$", "$\\theta^2$", "$\\theta^3$", "$\\theta^4$", 
                "$\\theta^5$", "$\\cos\\theta$", "$\\sin\\theta$", 
                "$\\sin^2\\theta$", "$\\cos^2\\theta$","$\\dot\\theta$", 
                "$\\dot\\theta^2$", "$\\dot\\theta^3$", "$\\dot\\theta^4$", 
                "$\\dot\\theta^5$", "$\\cos\\dot\\theta$", "$\\sin\\dot\\theta$", 
                "$\\sin^2\\dot\\theta$", "$\\cos^2\\dot\\theta$",
                "$\\theta\\dot\\theta$", "$\\theta^2\\dot\\theta$", 
                "$\\theta\\dot\\theta^2$", "$\\theta^2 (\\dot\\theta)^2$", 
                "$(\\theta^3)*(\\dot\\theta)$", "$(\\theta)*(\\dot\\theta^3)$", 
                "$\\theta^2 \\dot\\theta^3$", "$(\\theta^3)*(\\dot\\theta^2)$",
                "$\\cos(\\theta*\\dot\\theta)$", "$\\sin(\\theta*\\dot\\theta)$"))
my_xticks[0:21]=tmp[0:21]
#print(my_xticks)
plt.figure(figsize=(3.2*3,3))
plt.plot(np.abs(tval[0:21]), 'o',label="true",fillstyle='none')
plt.plot(np.abs(lasso[0:21]), '^', label="lasso",fillstyle='none')
plt.plot(np.abs(el9[0:21]), '*', label="elnet $\\alpha=0.9$")
plt.plot(np.abs(el5[0:21]), 'p',label="elnet $\\alpha=0.5$",fillstyle='none')
plt.plot(np.abs(ridge[0:21]), '>', label="ridge",fillstyle='none')
plt.plot(np.abs(ls[0:21]), '.', label="least squares")
plt.xticks(np.arange(tval.size)[0:21],my_xticks[0:21],rotation=75);
plt.ylabel("|coefficient|")
plt.ylim([0,4])
plt.legend(loc="best")
show("pendulum_coeff_noise.pdf")


plt.figure(figsize=(3.2*3,3))
plt.plot(np.abs(tval[0:21]), 'o',label="true")
plt.plot(np.abs(lasso[0:21]), '^', label="lasso")
plt.plot(np.abs(el9[0:21]), '*', label="elnet $\\alpha=0.9$")
plt.plot(np.abs(el5[0:21]), 'p',label="elnet $\\alpha=0.5$")
plt.plot(np.abs(ridge[0:21]), '>', label="ridge")
plt.semilogy(np.abs(ls[0:21]), '.', label="least squares")
plt.xticks(np.arange(tval.size)[0:21],my_xticks[0:21],rotation=75);
plt.ylabel("|coefficient|")
plt.ylim([1e-4,30])
plt.legend(loc="best")
show("pendulum_coeff_log_noise.pdf")

In [ ]:
def get_sample_small(runs,points):
    y_vect = np.zeros(runs*points)
    x_mat = np.zeros((runs*points, 28))
    basis_dict = lambda pos, vel: np.array([pos, pos**2, pos**3, pos**4, pos**5, 0*pos**6, 0*pos**7, math.sin(pos)**2, math.cos(pos)**2,
                                            vel, vel**2, vel**3, vel**4, vel**5, math.cos(vel), math.sin(vel), math.sin(vel)**2, math.cos(vel)**2,
                                            pos*vel, (pos**2)*vel, pos*vel**2, (pos**2)*(vel**2), (pos**3)*(vel), (pos)*(vel**3), (pos**2)*(vel**3), (pos**3)*(vel**2),
                                            math.cos(pos*vel), math.sin(pos*vel)])
    count = 0
    
    for run in range(runs):
        init_vel = random.uniform(a=-0.01,b=0.01)
        init_pos = random.uniform(a=-np.pi/100, b=np.pi/100)
        solution = solve_ivp(fun=RHS, y0=[init_vel, init_pos], t_span=[0,25], max_step=0.01, method="RK45")
        steps = solution.t.size
        chosen_points = np.random.randint(low=1, high=steps-1,size=points)
        velocity = solution.y[0,:].copy()
        position = solution.y[1,:].copy()
        for point in range(points):
            y_vect[count] = ((velocity[chosen_points[point]] - velocity[chosen_points[point]-1])/
                      (solution.t[chosen_points[point]] - solution.t[chosen_points[point]-1]))
            x_mat[count,:] = basis_dict(position[chosen_points[point]],
                                        velocity[chosen_points[point]])
            count += 1
    return y_vect, x_mat

In [ ]:
y,X = get_sample_small(10,100)

In [ ]:
plt.scatter(y, -b/m*X[:,9]- g/l*X[:,6])
plt.scatter(y, -b/m*X[:,9]-g/l*X[:,0]+X[:,2]/6*g/l - g/l/120*X[:,4])
plt.scatter(y, -b/m*X[:,9]-g/l*X[:,0]+X[:,2]/6*g/l - 0*g/l/120*X[:,4])
plt.scatter(y, -b/m*X[:,9]-g/l*X[:,0]+0*X[:,2]/6*g/l - 0*g/l/120*X[:,4])
plt.plot([np.min(y),np.max(y)],[np.min(y),np.max(y)])

In [ ]:
from sklearn.linear_model import LassoCV
reg = LassoCV(cv=5, random_state=0, normalize=True).fit(X, y)
print(reg.score(X, y) )
print("Sin coeff should be", -g/l, "it is",reg.coef_[6])
print("pos coeff should be", -g/l, "it is",reg.coef_[0])
print("pos^3 coeff should be", g/l/6, "it is",reg.coef_[2])
print("pos^5 coeff should be", -g/l/120, "it is",reg.coef_[4])
print("Velocity coeff should be", -b/m, "it is",reg.coef_[9])
reg.coef_

In [ ]:
reg = ElasticNetCV(cv=5, random_state=0, normalize=True, l1_ratio=0.8).fit(X, y)
print(reg.score(X, y) )
print("Sin coeff should be", -g/l, "it is",reg.coef_[6])
print("pos coeff should be", -g/l, "it is",reg.coef_[0])
print("pos^3 coeff should be", g/l/6, "it is",reg.coef_[2])
print("pos^5 coeff should be", -g/l/120, "it is",reg.coef_[4])
print("Velocity coeff should be", -b/m, "it is",reg.coef_[9])
reg.coef_

In [ ]:
def get_sample_small_noise(runs,points, noise=0.0001):
    y_vect = np.zeros(runs*points)
    x_mat = np.zeros((runs*points, 28))
    basis_dict = lambda pos, vel: np.array([pos, pos**2, pos**3, pos**4, pos**5, math.cos(pos), 0*math.sin(pos), math.sin(pos)**2, math.cos(pos)**2,
                                            vel, vel**2, vel**3, vel**4, vel**5, math.cos(vel), math.sin(vel), math.sin(vel)**2, math.cos(vel)**2,
                                            pos*vel, (pos**2)*vel, pos*vel**2, (pos**2)*(vel**2), (pos**3)*(vel), (pos)*(vel**3), (pos**2)*(vel**3), (pos**3)*(vel**2),
                                            math.cos(pos*vel), math.sin(pos*vel)])
    count = 0
    
    for run in range(runs):
        init_vel = random.uniform(a=-0.01,b=0.01)
        init_pos = random.uniform(a=-np.pi/10, b=np.pi/10)
        solution = solve_ivp(fun=RHS, y0=[init_vel, init_pos], t_span=[0,25], max_step=0.01, method="RK45")
        steps = solution.t.size
        chosen_points = np.random.randint(low=1, high=steps-1,size=points)
        velocity = solution.y[0,:] + np.random.normal(loc=0, scale=noise, size=steps)
        position = np.arctan2(np.sin(solution.y[1,:]),np.cos(solution.y[1,:])) + np.random.normal(loc=0, scale=noise, size=steps)
        for point in range(points):
            y_vect[count] = ((velocity[chosen_points[point]] - velocity[chosen_points[point]-1])/
                      (solution.t[chosen_points[point]] - solution.t[chosen_points[point]-1]))
            x_mat[count,:] = basis_dict(position[chosen_points[point]],
                                        velocity[chosen_points[point]])
            count += 1
    return y_vect, x_mat

In [ ]:
y,X = get_sample_small_noise(100,10)

In [ ]:
#plt.scatter(y, -b/m*X[:,8]- g/l*X[:,5])
#plt.scatter(X[:,0],X[:,1],s=1)
plt.figure(figsize=(8,8))
plt.subplot(331)
plt.scatter(X[:,0], y, s=1, marker=".", c="k")
plt.ylabel("$\\ddot{\\theta}$")
plt.xlabel("$\\theta$")
plt.subplot(332)
plt.scatter(X[:,2], y, s=1, marker=".", c="k")
plt.xlabel("$\\theta^3$")
plt.subplot(333)
plt.scatter(X[:,4], y, s=1, marker=".", c="k")
plt.xlabel("$\\theta^5$")

#row 2
plt.subplot(334)
plt.scatter(X[:,9], y, s=1, marker=".", c="k")
plt.ylabel("$\\ddot{\\theta}$")
plt.xlabel("$\\dot\\theta$")
plt.subplot(335)
plt.scatter(X[:,10], y, s=1, marker=".", c="k")
plt.xlabel("$\\dot{\\theta}^2$")
plt.subplot(336)
plt.scatter(X[:,19], y, s=1, marker=".", c="k")
plt.xlabel("$\\theta^2\\dot{\\theta}$")
#row 3
plt.subplot(337)
plt.scatter(b/m*X[:,9]+g/l*X[:,0], y, s=1, marker=".", c="k")
plt.ylabel("$\\ddot{\\theta}$")
plt.xlabel("$\\frac{b}{m}\\dot\\theta + \\frac{g}{L}\\theta $")
plt.subplot(338)
plt.scatter(b/m*X[:,9]+g/l*X[:,0]-g/l/6*X[:,2], y, s=1, marker=".", c="k")
plt.xlabel("$\\frac{b}{m}\\dot\\theta+\\frac{g }{L}\\theta-\\frac{g }{6 L}\\theta^3$")
plt.subplot(339)
plt.scatter(b/m*X[:,9]+g/l*X[:,0]-g/l/6*X[:,2]+g*X[:,4]/120/l, y, s=1, marker=".", c="k")
plt.xlabel("$\\frac{b}{m}\\dot\\theta-\\frac{g \\theta}{L}+\\frac{g \\theta^3}{6 L}-\\frac{g \\theta^5}{120 L}$")
plt.tight_layout()
#plt.subplots_adjust(bottom=0.9,top=1) #, right=1.8, top=0.9)
show("pendulum_scatter_small.pdf")

In [ ]:
from sklearn.linear_model import LassoCV
reg = LassoCV(cv=5, random_state=0, normalize=True).fit(X, y)
print(reg.score(X, y) )
print("Sin coeff should be", -g/l, "it is",reg.coef_[6])
print("pos coeff should be", -g/l, "it is",reg.coef_[0])
print("pos^3 coeff should be", g/l/6, "it is",reg.coef_[2])
print("pos^5 coeff should be", -g/l/120, "it is",reg.coef_[4])
print("Velocity coeff should be", -b/m, "it is",reg.coef_[9])
reg.coef_

In [ ]:
reg = ElasticNetCV(cv=5, random_state=0, normalize=True, l1_ratio=0.9).fit(X, y)
print(reg.score(X, y) )
print("Sin coeff should be", -g/l, "it is",reg.coef_[6])
print("pos coeff should be", -g/l, "it is",reg.coef_[0])
print("pos^3 coeff should be", g/l/6, "it is",reg.coef_[2])
print("pos^5 coeff should be", -g/l/120, "it is",reg.coef_[4])
print("Velocity coeff should be", -b/m, "it is",reg.coef_[9])
reg.coef_

Run several pendula of different masses, lengths, and damping forces

In [ ]:
pendula = 10

xdata = np.zeros((pendula,6))
ydata = np.zeros((pendula,2))
ydata_el = np.zeros((pendula,2))

for p in range(pendula):
    b = np.random.uniform(low=0.01,high=0.5)
    l = np.random.uniform(low=0.24,high=4)
    g = 9.81
    m = np.random.uniform(low=0.01,high=0.5)

    y,X = get_sample_with_noise(100,10,0.01)

    reg = LassoCV(cv=5, random_state=0, normalize=True).fit(X, y)
    reg.score(X, y) 
    print("Sin coeff should be", -g/l, "it is",reg.coef_[6])
    print("Velocity coeff should be", -b/m, "it is",reg.coef_[9])
    ydata[p,:] = (reg.coef_[6],reg.coef_[9])
    

    reg = ElasticNetCV(cv=5, random_state=0, normalize=True, l1_ratio=0.9).fit(X, y)
    reg.score(X, y) 
    print("Sin coeff should be", -g/l, "it is",reg.coef_[6])
    print("Velocity coeff should be", -b/m, "it is",reg.coef_[9])
    ydata_el[p,:] = (reg.coef_[6],reg.coef_[9])
    xdata[p,:] = [g/l,b**2/m**2,(g/b*m/l)**2,b/m,np.sqrt(g/l),g/b*m/l]

In [ ]:
reg_coef = LassoCV(cv=5, random_state=0, normalize=True).fit(xdata[:,0:3],ydata[:,0])
print(reg_coef.coef_)

reg_coef = LassoCV(cv=5, random_state=0, normalize=True).fit(xdata[:,3:6],ydata[:,1])
print(reg_coef.coef_)